# Table of Contents
* [Stage 4b](#Stage-4b)
* [process data](#process-data)
* [analysis](#analysis)
* [compute consensus](#compute-consensus)
* [bounding box clustering viz](#bounding-box-clustering-viz)
* [hide](#hide)


In [4]:
%%capture
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

import matplotlib as mpl
mpl.use("Agg")
import matplotlib.pylab as plt
#%matplotlib notebook
%matplotlib inline
%load_ext base16_mplrc
%base16_mplrc light default
plt.rcParams['figure.figsize'] = (16.0, 10.0)

import re
import pickle
import boto
from copy import deepcopy
import json
import os
import random
import jinja2
from tqdm import tqdm
from IPython.core.display import HTML

import PIL.Image as Image
import requests

from boto.mturk.qualification import PercentAssignmentsApprovedRequirement, Qualifications, Requirement

from keysTkingdom import mturk_ai2
from keysTkingdom import aws_tokes
from keysTkingdom import mturk_aristo

from anigen_tools.mturk import pickle_this, unpickle_this

import os
import jinja2
import argparse
from jinja2 import Environment, FileSystemLoader

j2env = jinja2.Environment()

from anigen_tools.bboxes import cluster_from_nms
from anigen_tools.bboxes import draw_animation_seq
from anigen_tools.bboxes import cluster_from_annos
from anigen_tools.bboxes import create_subtask_data
from anigen_tools.bboxes import draw_image_and_labels
from anigen_tools.bboxes import cluster_and_label
from anigen_tools.bboxes import cluster_and_label_stage_1b
from anigen_tools.bboxes import cluster_from_nms_stage1_b

import warnings
warnings.filterwarnings('ignore')

# Stage 4b

first batch:

In [5]:
# test_batch_2 = unpickle_this('test_batch_4b_2.pkl')
# test_batch_2_alt = unpickle_this('test4b_2.pkl')
# full_batch_2 = unpickle_this('full_batch_2_update.pkl')

# new_4b_assignemnts = unpickle_this('additional_batch_1_4b_update_10_8.pkl')

# new_4b_assignemnts = unpickle_this('additional_batch_1_4b_update_10_10.pkl')

# new_4b_assignemnts = unpickle_this('additional_batch_1_4b_update_10_10.pkl')

In [1]:
new_4b_prod_2_3_4 = unpickle_this('new_4b_assignemnts_10_26.pkl')

NameError: name 'unpickle_this' is not defined

In [39]:
batch_2_results_update =[]
for assignment_triple in new_4b_assignemnts.values():
    if len(assignment_triple) > 2:
        batch_2_results_update.extend(assignment_triple)

# process data

In [41]:
def create_result(assmt):
    result = json.loads(assmt.answers[0][0].fields[0])
    result['h_id'] = assmt.HITId
    result['a_id'] = assmt.AssignmentId 
    result['worker_id'] = assmt.WorkerId
    return result

In [42]:
batch_2_results = [create_result(ar) for ar in batch_2_results_update]
objects_present = [{'h_id': anno['h_id'], 'w_id': anno['worker_id'], 'still_id': anno['stillID'], 'objects': set([ch['label'] for ch in json.loads(anno['characterBoxes'])])} for anno in batch_2_results]

assignment_results = batch_2_results
annotations_by_frame = defaultdict(list)
for anno in sorted(assignment_results, key=lambda x: x['stillID']):
    animation_id = anno['stillID']
    annotations_by_frame[animation_id].append(anno)

len(annotations_by_frame)

879

## analysis

In [25]:
s4b_df = pd.DataFrame(objects_present)
s4b_df['aid'] = s4b_df['still_id'].apply(lambda x: x.rsplit('_', 1)[0])

In [28]:
# bbox_df['bbox'] = bbox_df['raw_bbox'].apply(lambda z: characterbox_to_box(next(iter(sorted(z, key= lambda x: x['left']) or []), {"left":0,"top":0,"width":0,"height":0})))

In [29]:
# bbox_df['frame_n'] = bbox_df['bbox'].apply(lambda x: choose_frame(x))

In [33]:
def choose_frame(bbox):
    if bbox[0] < 640:
        return '_10.png'
    else:        
        return '_70.png'

def assign_con_box(row):
    first_frame = row['frame_n'] == '_10.png'
    aid = row['still_id'][:-10]
    if aid in subtask_consensus_boxes:
        con_box_choices = subtask_consensus_boxes[aid]
    else:
        return([0, 0, 0, 0])
    for box in con_box_choices:
        if box['label'] in row['characters']:
            if box['box'][0] < 640 and first_frame:
                return box['box'].tolist()
            if box['box'][0] > 640 and not first_frame:
                return box['box'].tolist()
    return([1, 0, 0, 0])

In [30]:
# bbox_df['con_box'] = bbox_df.apply(assign_con_box, axis=1)

In [31]:
# char_change = defaultdict(list)
# for aid, annos in list(subtask_by_frame.items()):
#     disapear_count = 0
#     for anno in annos:
#         la = json.loads(anno['characterBoxes'])
#         if len(la) !=2:
#             disapear_count += 1
#     if disapear_count == 3:
#         char_change[aid].append(la)

# compute consensus

In [43]:
from amt_utils.bboxes import draw_object_seq
from amt_utils.bboxes import comp_box_center
from amt_utils.bboxes import comp_boxes_iou
from amt_utils.bboxes import characterbox_to_box
from amt_utils.bboxes import rect_from_anno
from amt_utils.bboxes import box_area
from amt_utils.bboxes import is_duplicate
from amt_utils.bboxes import filter_keep_by_area_fraction
from amt_utils.bboxes import assign_boxes
from amt_utils.bboxes import box_area

In [44]:
def assign_frame(box):
    frame_width = 640
    box_center = comp_box_center(box)
    return int(box_center[0] // frame_width)


def enclosing_box(boxes):
    rects = np.array([box['box'] for box in boxes])
    lower = rects[:,0:2].min(axis=0)
    upper = rects[:,2:].max(axis=0)
    enclosing_rect = np.concatenate([lower, upper])
    new_box = {
        'box': enclosing_rect,
        'area': box_area(enclosing_rect.reshape(2, 2)),
        'label': boxes[0]['label'],
        'idx': 0,
        'frame': boxes[0]['frame']
    }
    return new_box


def nms(boxes, thresh):
    boxes = sorted(boxes, key=lambda x: x['area'], reverse=True)  # largest area first
    keep = [None] * len(boxes)
    for i, box in enumerate(boxes):
        keep[i] = not is_duplicate(i, boxes, thresh)
    filter_keep_by_area_fraction(boxes, keep, thresh)
    selected_boxes = [boxes[i] for i in range(len(boxes)) if keep[i]]
    duplicate_boxes = [boxes[i] for i in range(len(boxes)) if not keep[i]]
    assign_boxes(selected_boxes, duplicate_boxes)
    for b1 in selected_boxes:
        idx = b1['idx']
        votes = 1
        for b2 in duplicate_boxes:
            if b2['duplicate_of'] == idx:
                votes += 1

        b1['votes'] = votes
    if not duplicate_boxes:
        duplicate_boxes = []
    filtered_boxes = [box for box in selected_boxes if box['votes'] > 1]
    return filtered_boxes


def choose_object_boxes(object_annotations):
    single_turker_boxes = [json.loads(box['characterBoxes']) for box in object_annotations]
    single_turker_boxes = [b for sl in single_turker_boxes for b in sl ]
    boxes = [None] * len(single_turker_boxes)
    for i, charBox in enumerate(single_turker_boxes):
        box = characterbox_to_box(charBox)
        area = charBox['width'] * charBox['height']
        label = charBox['label']
        boxes[i] = {
            'box': box,
            'area': area,
            'label': label,
            'idx': i,
            'frame': assign_frame(box)
        }
    boxes = sorted(boxes, key=lambda x: x['area'])
    boxex_by_label = defaultdict(lambda : defaultdict(list))
    for box in boxes:
        if box['area'] < 100:
            continue
        boxex_by_label[box['label']][box['frame']].append(box)
    con_boxes = []
    for obj, frames_boxes in boxex_by_label.items():
        for frame_n, boxes in frames_boxes.items():
            con_box = nms(boxes, 0.2)
            if con_box:
                if len(con_box) < 2:
                    con_boxes.append(con_box[0])
                else:
                    con_boxes.append(enclosing_box(con_box))
            else:
                con_boxes.append(enclosing_box(boxes))
    return con_boxes

In [45]:
subtask_consensus_boxes = {}
for aid, animation_annos in tqdm(list(annotations_by_frame.items())):
    con_boxes = choose_object_boxes(animation_annos)
    subtask_consensus_boxes[aid] = con_boxes

100%|██████████| 879/879 [00:00<00:00, 1267.53it/s]


In [46]:
pickle_this(subtask_consensus_boxes, 'stage_4b_con_batch_2_latest_10_10.pkl')

# bounding box clustering viz

In [154]:
from amt_utils.bboxes import draw_object_seq
from amt_utils.bboxes import comp_box_center
from amt_utils.bboxes import characterbox_to_box

In [ ]:
rev_dir = 'review_results'
img_dir = 'review_frames'

In [142]:
# con_boxes, labels = cluster_and_label_stage_1b(animation_annos, cluster_from_nms_stage1_b, 1, 3, './subtask_data/frames/')

In [159]:
test_obj_anno = annotations_by_frame[problem_vid]

In [ ]:
# frame_origin = (0, 0)
# frame_size = (640, 480)
# frame_o2 = (640, 0)
# frame_ext = (1280, 480)

# subtask_rev_seq = []
# stage_1b_consensus_boxes = {}
# for aid, animation_annos in tqdm(list(subtask_by_frame.items())[:51]):
#     try:
#         middle_frame_annos = annotations_by_frame[aid][3:6]
#         full_original_annos = annotations_by_frame[aid]
#         two_frame_image, con_boxes, labels = draw_image_and_labels(animation_annos, cluster_from_nms, 1, 3, './subtask_data/frames/')
#         three_frame_img, consensus_boxes, labels_orig = draw_animation_seq(full_original_annos, cluster_from_nms)
#         single_frame_image, single_con_boxes, labels = draw_image_and_labels(middle_frame_annos, cluster_from_nms, 1, 3)
#         left_image = two_frame_image.crop(frame_origin + frame_size)
#         right_image = two_frame_image.crop(frame_o2 + frame_ext)
#         imgs_comb = Image.fromarray(np.hstack([left_image, single_frame_image, right_image]))
#         o_img_path = os.path.join(img_dir, aid) + '_rev.png'
#         new_img_path = os.path.join(img_dir, aid) + 'subtask_rev.png'
#         three_frame_img.save(os.path.join(rev_dir, o_img_path))
#         imgs_comb.save(os.path.join(rev_dir, new_img_path))
#         subtask_rev_seq.append((aid, o_img_path, new_img_path))
#     except:
#         print(aid)

In [131]:
review_seq = {}
error_frames = []
for aid, animation_annos in tqdm(list(annotations_by_frame.items())[:2]):
    try:
        three_frame_img, consensus_boxes, labels = draw_animation_seq(animation_annos, cluster_from_nms_stage1_b)
        avg_votes = np.mean([box['votes'] for frame in consensus_boxes for box in frame])
        possible_labels = [c['possible_labels'] for c in sorted(labels, key=lambda x: x['box'][0])]
        chosen_labels = [c['chosen_labels'] for c in sorted(labels, key=lambda x: x['box'][0])]
        review_seq[aid] = {'votes': avg_votes, 'possible_labels': possible_labels, 'chosen_labels': chosen_labels}
        img_path = os.path.join(rev_dir, img_dir, aid) + '_rev.png'
        three_frame_img.save(img_path)
    except IndentationError:
        error_frames.append(aid)
rev_page = generate_review_page('bbox_review_10_3.html')

  0%|          | 0/2 [00:00<?, ?it/s]

['stillID', 'characterBoxes', 'h_id', 'a_id', 'worker_id', 'stillID', 'characterBoxes', 'h_id', 'a_id', 'worker_id', 'stillID', 'characterBoxes', 'h_id', 'a_id', 'worker_id']


TypeError: string indices must be integers

In [127]:
three_frame_img, consensus_boxes, labels = draw_animation_seq(annotations_by_frame[problem_vid], cluster_from_nms)
avg_votes = np.mean([box['votes'] for frame in consensus_boxes for box in frame])
possible_labels = [c['possible_labels'] for c in sorted(labels, key=lambda x: x['box'][0])]
chosen_labels = [c['chosen_labels'] for c in sorted(labels, key=lambda x: x['box'][0])]
review_seq[aid] = {'votes': avg_votes, 'possible_labels': possible_labels, 'chosen_labels': chosen_labels}
three_frame_img

# hide

In [ ]:
# for vid in list(subtask_consensus_boxes.values()):
#     for box in vid:
#         if 'label' not in box.keys():
#             print(box)

In [32]:
# def choose_best_box(obj_boxes, thresh=0.05):
#     for i, b1 in enumerate(obj_boxes):
#         for j in range(i + 1, len(obj_boxes)):
#             b2 = obj_boxes[j]
#             print(b1, b2, comp_boxes_iou(b1['box'], b2['box']))
#             if comp_boxes_iou(b1['box'], b2['box']) > thresh:
#                 print('true')
# #                 return b1
#     return obj_boxes[0]

In [ ]:
# def choose_best_box(obj_boxes):
#     print(obj_boxes, '\n')
#     return obj_boxes[1]

In [ ]:
vote_series = pd.Series([vote for vote in review_seq.values()])

In [13]:
# assignments =[]
# for assignment_triple in list(results.values()):
#     assignments.extend(assignment_triple)

# assignment_results = [create_result(ar) for ar in assignments]

# characters_present = [{'h_id': anno['h_id'], 'w_id': anno['worker_id'], 'still_id': anno['stillID'], 'characters': set([ch['label'] for ch in json.loads(anno['characterBoxes'])])} for anno in assignment_results]

# main_characters =  {"fred",
#              "barney",
#               "wilma",
#               "betty",
#               "pebbles",
#               "bamm bamm",
#               "dino",
#               "mr slate",
#               "baby puss",
#               "hoppy",
#               "no characters"}

# def all_main(char_set, main_chars=main_characters):
#     return not bool(char_set.difference(main_chars))

# char_df = pd.DataFrame(characters_present)

# char_df['all_main'] = char_df['characters'].apply(lambda x: all_main(x))

# all_chars = [c for char_list in char_df['characters'].apply(lambda x: list(x)).tolist() for c in char_list]
# char_counts = pd.Series(all_chars).value_counts()

# # char_counts.hist(bins = 30, log=True)

# char_df['n_chars'] = char_df['characters'].apply(lambda x: len(x))

# prolific_workers = char_df['w_id'].value_counts().index.tolist()[:5]

# grouped_by_still = char_df.groupby('still_id')

# agg_df = grouped_by_still.aggregate(len)

# import scipy

# agged_on_mode = grouped_by_still.agg(lambda x: scipy.stats.mode(x))
# consensus_results_df = agged_on_mode.reset_index()

# consensus_results_df['count_mode'] = consensus_results_df['n_chars'].apply(lambda x: x[1][0])

# consensus_results_df

# consensus_results_df['mode_count'] = consensus_results_df['characters'].apply(lambda x: x[1][0])

# consensus_results_df['all_agree'] = consensus_results_df['count_mode'] == 3
# consensus_results_df['two_agree'] = consensus_results_df['count_mode'] == 2

In [87]:
# all_main_annotations_by_frame = {}
# no_main_annotations_by_frame = {}
# mixed_all_frames = {}


# for aid, annos in annotations_by_frame.items():
#     still_ids = [still['stillID'] for still in annos]
#     annos = [anno for anno in annos]
#     if len([sid for sid in still_ids if sid in all_agree_set]) == 9:
#         all_main_annotations_by_frame[aid] = annos
#     elif len([sid for sid in still_ids if sid in not_main]) == 9:
#         no_main_annotations_by_frame[aid] = annos
#     else: 
#         mixed_all_frames[aid] = annos

# print(len(all_main_annotations_by_frame), len(no_main_annotations_by_frame), len(mixed_all_frames))

In [14]:
# char_count_df = consensus_results_df[consensus_results_df['count_mode'] == 3]

# # consensus_results_df[agree_all_main]['characters'].apply(lambda x: x[1][0]).value_counts()

# not_main =set(consensus_results_df[~agree_all_main]['still_id'].tolist())

# not_main_df = char_df[char_df['still_id'].isin(not_main)]

# consensus_results_df.head(1)

# agree_all_main = consensus_results_df['all_main'].apply(lambda x: x[1][0]) == 3 

# all_agree = consensus_results_df[agree_all_main][consensus_results_df[agree_all_main]['all_agree']]
# disagree = consensus_results_df[agree_all_main][~consensus_results_df[agree_all_main]['all_agree']]







# all_agree_set = set(all_agree['still_id'].tolist())

# count_all_agree_set = set(consensus_results_df[consensus_results_df['count_mode'] == 3]['still_id'].tolist())
# count_two_agree_set = set(consensus_results_df[consensus_results_df['count_mode'] == 2]['still_id'].tolist())
# count_none_agree_set = set(consensus_results_df[consensus_results_df['count_mode'] == 1]['still_id'].tolist())

# conflict_images = set(disagree['still_id'].tolist())

# conflict_df = char_df[char_df['still_id'].isin(conflict_images)]

# import PIL.Image as Image
# import requests

# image_n = 10

# consensus_results_df.iloc[image_n * 9: image_n * 9 + 3]

# s3_base_path = 'https://s3-us-west-2.amazonaws.com/ai2-vision-animation-gan/annotation_data/still_frames/'

# image_url = s3_base_path + consensus_results_df['still_id'].iloc[image_n *  9]
# Image.open(requests.get(image_url, stream=True).raw)